In [1]:
import os
import sys
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image, input_data, datasets
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
raw_dir = params.raw_dir
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
#subjects_to_skip = ['sub-025', 'sub-038']
subjects_to_skip = []
subs = sub_info[(sub_info['group'] == 'control') & (~sub_info['sub'].isin(subjects_to_skip))]['sub'].tolist()

run_num = 3
runs = list(range(1, run_num + 1))

def apply_glasser_atlas(subject_data, atlas_img):
    # Resample atlas to subject space
    resampled_atlas = image.resample_to_img(atlas_img, subject_data, interpolation='nearest')
    
    # Extract time series data
    atlas_masker = input_data.NiftiLabelsMasker(labels_img=resampled_atlas, 
                                               standardize=True,
                                               memory='nilearn_cache', 
                                               verbose=0)
    time_series = atlas_masker.fit_transform(subject_data)
    
    return time_series

def create_connectivity_matrix(ss):
    logging.info(f"Processing subject: {ss}")
    
    # Download and load Glasser atlas
    atlas = datasets.fetch_atlas_glasser_2016()
    atlas_img = atlas.maps
    
    all_runs_data = []
    
    for rn in runs:
        # Load subject data for this run
        run_path = f'{raw_dir}/{ss}/ses-01/derivatives/fsl/loc/run-0{rn}/1stLevel.feat/filtered_func_data_reg.nii.gz'
        if not os.path.exists(run_path):
            logging.warning(f'Run data not found: {run_path}')
            continue
        
        subject_img = nib.load(run_path)
        
        # Apply Glasser atlas and extract time series
        run_data = apply_glasser_atlas(subject_img, atlas_img)
        all_runs_data.append(run_data)
    
    if not all_runs_data:
        logging.warning(f'No valid run data found for subject {ss}')
        return None
    
    # Concatenate runs
    full_time_series = np.concatenate(all_runs_data, axis=0)
    
    # Compute connectivity matrix
    correlation_measure = ConnectivityMeasure(kind='correlation')
    connectivity_matrix = correlation_measure.fit_transform([full_time_series])[0]
    
    return connectivity_matrix

def calculate_group_matrices():
    all_matrices = []
    
    for ss in subs:
        matrix_path = f'{results_dir}/connectivity_matrices/{ss}_connectivity_matrix.npy'
        if os.path.exists(matrix_path):
            matrix = np.load(matrix_path)
            all_matrices.append(matrix)
    
    if all_matrices:
        # Calculate mean and std matrices
        all_matrices = np.array(all_matrices)
        mean_matrix = np.mean(all_matrices, axis=0)
        std_matrix = np.std(all_matrices, axis=0)
        
        # Save group-level matrices
        output_dir = f'{results_dir}/connectivity_data'
        os.makedirs(output_dir, exist_ok=True)
        np.save(f'{output_dir}/glasser_group_mean_matrix.npy', mean_matrix)
        np.save(f'{output_dir}/glasser_group_std_matrix.npy', std_matrix)
        logging.info('Saved group-level matrices')

def main():
    # Create individual connectivity matrices
    for ss in subs:
        connectivity_matrix = create_connectivity_matrix(ss)
        if connectivity_matrix is not None:
            # Save the connectivity matrix
            output_dir = f'{results_dir}/connectivity_matrices'
            os.makedirs(output_dir, exist_ok=True)
            np.save(f'{output_dir}/{ss}_glasser_connectivity_matrix.npy', connectivity_matrix)
            logging.info(f'Saved connectivity matrix for {ss}')
    
    # Calculate and save group-level matrices
    calculate_group_matrices()

if __name__ == "__main__":
    main()

/home/csimmon2/anaconda3/envs/brainiak_env/lib/python3.7/site-packages/nilearn/__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()
/home/csimmon2/anaconda3/envs/brainiak_env/lib/python3.7/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)
2024-10-25 12:47:20,082 - INFO - Processing subject: sub-057


AttributeError: module 'nilearn.datasets' has no attribute 'fetch_atlas_glasser_2016'